# Read Overpass-API to Python Pandas Dataframe

The [Overpass API](http://wiki.openstreetmap.org/wiki/Overpass_API) provides access to the data behind the Openstreetmaps Map Data.
The [Overpass-Turbo](http://overpass-turbo.eu/) is the easyiest way to test requests and get the correct code to ask the database. Example: Fountains of the city of Rome as per https://github.com/water-fountains/proximap/issues/133

In [1]:
from datetime import datetime as dt
dtFmt = "%y%m%d_%H%M%S"
print (dt.now().strftime(dtFmt))
import pandas as pd
import requests
import json
pd.set_option('display.max_columns', 200)

191201_082130


In [2]:
# Get yours at: http://boundingbox.klokantech.com/
# Rome ll: 41.8020,12.3170 ; ur: 41.9973,12.6219
bbox = [12.3170,41.8020,12.6219,41.9973]

# Links unten
minLat = bbox[1]
minLon = bbox[0]

# Rechts oben
maxLat = bbox[3]
maxLon = bbox[2]

### Construct the Overpass Query String

Request from [Overpass-Turbo](http://overpass-turbo.eu/)

In [3]:
bbox_string = '(%s,%s,%s,%s)' % (minLat, minLon, maxLat, maxLon)

compactOverpassQLstring = f'[out:json][timeout:60];(node[man_made=drinking_fountain]{bbox_string};node[amenity=drinking_water]{bbox_string};node[amenity=fountain]{bbox_string};'
#node[man_made=water_tap]{bbox_string};node[natural=spring]{bbox_string};node[amenity=watering_place]{bbox_string};
#node[amenity=water_point]{bbox_string};node[man_made=water_well]{bbox_string};
compactOverpassQLstring += ');out ;'
compactOverpassQLstring

'[out:json][timeout:60];(node[man_made=drinking_fountain](41.802,12.317,41.9973,12.6219);node[amenity=drinking_water](41.802,12.317,41.9973,12.6219);node[amenity=fountain](41.802,12.317,41.9973,12.6219););out ;'

In [4]:
osmrequest = {'data': compactOverpassQLstring}
osmurl = 'https://overpass-api.de/api/interpreter'

# Ask the API
osm = requests.get(osmurl, params=osmrequest)

### Reformat the JSON to fit in a Pandas Dataframe

The JSON can't be directyl imported to a Pandas Dataframe:

Thanks to [unutbu from stackoverflow.com](http://stackoverflow.com/questions/24848416/expected-string-or-unicode-when-reading-json-with-pandas) for fiddling this out!

In [5]:
osmdata = osm.json()
# print out the json
osmdata

{'version': 0.6,
 'generator': 'Overpass API 0.7.55.1009 5e627b63',
 'osm3s': {'timestamp_osm_base': '2019-12-01T07:36:02Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 246569213,
   'lat': 41.8245081,
   'lon': 12.485546,
   'tags': {'amenity': 'drinking_water', 'fountain': 'nasone'}},
  {'type': 'node',
   'id': 246569214,
   'lat': 41.8526205,
   'lon': 12.4783285,
   'tags': {'amenity': 'drinking_water', 'fountain': 'nasone'}},
  {'type': 'node',
   'id': 246569215,
   'lat': 41.8541022,
   'lon': 12.4768335,
   'tags': {'amenity': 'drinking_water', 'fountain': 'nasone'}},
  {'type': 'node',
   'id': 246569216,
   'lat': 41.8637486,
   'lon': 12.4789476,
   'tags': {'amenity': 'drinking_water', 'fountain': 'nasone'}},
  {'type': 'node',
   'id': 246571139,
   'lat': 41.9042869,
   'lon': 12.5132783,
   'tags': {'amenity': 'drinking_water', 'created_by': 'JOSM'}},


In [6]:
osmelements = osmdata['elements']
for dct in osmelements:
    for key, val in dct['tags'].items():
        dct[key] = val
    del dct['tags']

### Now put everything to the Pandas Dataframe

In [7]:
osmdf = pd.DataFrame(osmelements)

### Look at the whole Dataframe

In [8]:
osmdf.head(5)

,type,id,lat,lon,amenity,fountain,created_by,wheelchair,flow,architect,description,drinking_water,start_date,wetap:photo,wetap:status,wetap:statusnote,source,image,name,functional,capacity,tap,historic,historic:civilization,disused,tourism,wikidata,wikipedia,name:it,name:ru,indoor,access,disused:amenity,fee,note,bottle,dog,man_made,opening_hours,odbl,ref,addr:postcode,addr:street,highway,construction_date,inscription:it,name:en,noref,alt_name,flickr,fountain:type,name:de,name:es,name:fr,name:zh,survey:date,fixme,drinking_water:type,name:ja,name:sv,addr:city,addr:country,addr:housenumber,wheelchair:description,height,mapillary,operator,designation,level,location,covered,description:de,website,artwork_type,subject:wikidata,drinking_water:legal
0,node,246569213,41.824508,12.485546,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,246569214,41.852621,12.478328,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,246569215,41.854102,12.476833,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,246569216,41.863749,12.478948,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,246571139,41.904287,12.513278,drinking_water,NaN,JOSM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
osmdf.tail(5)

,type,id,lat,lon,amenity,fountain,created_by,wheelchair,flow,architect,description,drinking_water,start_date,wetap:photo,wetap:status,wetap:statusnote,source,image,name,functional,capacity,tap,historic,historic:civilization,disused,tourism,wikidata,wikipedia,name:it,name:ru,indoor,access,disused:amenity,fee,note,bottle,dog,man_made,opening_hours,odbl,ref,addr:postcode,addr:street,highway,construction_date,inscription:it,name:en,noref,alt_name,flickr,fountain:type,name:de,name:es,name:fr,name:zh,survey:date,fixme,drinking_water:type,name:ja,name:sv,addr:city,addr:country,addr:housenumber,wheelchair:description,height,mapillary,operator,designation,level,location,covered,description:de,website,artwork_type,subject:wikidata,drinking_water:legal
1640,node,6893481812,41.913437,12.461680,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1641,node,6898351366,41.902114,12.590335,drinking_water,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pubblica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1642,node,6942907549,41.888007,12.461600,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643,node,6960516844,41.903375,12.512719,drinking_water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1644,node,7021581874,41.877493,12.474922,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Export to CSV

In [20]:
osmCsvFileName = "osmFountainsRome"+dt.now().strftime(dtFmt)+".csv"
osmdf.to_csv(osmCsvFileName, index=False)

In [21]:
print("wrote '"+osmCsvFileName+"' with "+str(len(osmdf))+" lines")

wrote 'osmFountainsRome191201_082705.csv' with 1645 lines
